本コンペは日次の野菜卸売価格の予測タスクですが、ここでは簡単のため、月次の予測タスクに読み替えてみます（日毎に予測値は変えない）<br>
本notebookでは前処理を行い、次のnotebookでは学習・推論を行います

In [1]:
# コンペデータを格納しているパスを指定ください
PATH = 'data'

In [2]:
cd {PATH}

/Users/takuma/Documents/kaggle/vegetable/data


In [3]:
import pandas as pd
import numpy as np

In [4]:
train_df = pd.read_csv('train.csv')
print(train_df.shape)
train_df

(77751, 7)


,kind,date,amount,mode_price,area,year,weekno
0,だいこん,20051107,201445,735.0,千葉,2005,45
1,だいこん,20051108,189660,840.0,千葉_各地_青森,2005,45
2,だいこん,20051110,218166,735.0,千葉_各地_青森,2005,45
3,だいこん,20051111,182624,682.5,千葉_青森,2005,45
4,だいこん,20051112,220691,682.5,千葉_青森,2005,45
...,...,...,...,...,...,...,...
77746,しめじ,20170526,17231,48.5,新潟_長野,2017,21
77747,しめじ,20170527,17105,48.5,新潟_長野,2017,21
77748,しめじ,20170529,12406,48.5,新潟_長野,2017,22
77749,しめじ,20170530,12320,48.5,新潟_長野,2017,22


In [5]:
test_df = pd.read_csv('test.csv')
print(test_df.shape)
test_df

(315, 5)


,kind,date,area,year,weekno
0,だいこん,20221101,千葉_各地_神奈川,2022,44
1,だいこん,20221102,千葉_各地_神奈川,2022,44
2,だいこん,20221104,千葉_各地_神奈川,2022,44
3,だいこん,20221105,千葉_各地_神奈川,2022,44
4,だいこん,20221107,千葉_各地_神奈川,2022,45
...,...,...,...,...,...
310,ミニトマト,20221124,各地_愛知,2022,47
311,ミニトマト,20221125,各地_愛知,2022,47
312,ミニトマト,20221126,各地_愛知,2022,47
313,ミニトマト,20221128,各地_愛知,2022,48


In [6]:
wea_df = pd.read_csv('weather.csv')
print(wea_df.shape)
wea_df

(210336, 10)


,date,mean_temp,max_temp,max_temp_time,min_temp,min_temp_time,sum_rain,sun_time,mean_humid,area
0,20041106,13.1,20.1,2004/11/6 12:50,8.1,2004/11/6 05:31,5.0,1.6,77.0,青森
1,20041107,9.6,12.5,2004/11/7 13:40,5.4,2004/11/7 22:17,0.0,4.1,63.0,青森
2,20041108,9.0,15.5,2004/11/8 12:51,3.2,2004/11/8 06:28,0.0,8.9,72.0,青森
3,20041109,12.2,17.7,2004/11/9 14:40,5.7,2004/11/9 02:49,1.5,1.2,85.0,青森
4,20041110,11.6,16.4,2004/11/10 14:42,6.9,2004/11/10 06:23,0.0,7.5,89.0,青森
...,...,...,...,...,...,...,...,...,...,...
210331,20221027,12.8,17.7,2022/10/27 12:34,8.5,2022/10/27 05:35,0.0,6.1,65.0,東京
210332,20221028,13.5,19.2,2022/10/28 14:12,9.9,2022/10/28 05:21,0.0,3.7,73.0,東京
210333,20221029,15.8,22.0,2022/10/29 12:43,10.8,2022/10/29 04:12,0.0,8.3,59.0,東京
210334,20221030,13.8,19.2,2022/10/30 13:56,10.2,2022/10/30 06:22,0.0,7.7,58.0,東京


まず、天候データを加工します。月毎の基本統計量をとります

In [7]:
wea_df['year'] = wea_df['date'].apply(lambda x: int(str(x)[:4]))
wea_df['month'] = wea_df['date'].apply(lambda x: int(str(x)[4:6]))

In [10]:
agg_cols = ['mean_temp', 'max_temp', 'min_temp', 'sum_rain', 'sun_time', 'mean_humid']
gb_df = wea_df.groupby(['area', 'year', 'month'])[agg_cols].agg(['mean','max','min']).reset_index()

new_cols = []
for col1, col2 in gb_df.columns:
    if col2:
        new_cols.append(col2+'_'+col1)
    else:
        new_cols.append(col1)

gb_df.columns = new_cols
gb_df

,area,year,month,mean_mean_temp,max_mean_temp,min_mean_temp,mean_max_temp,max_max_temp,min_max_temp,mean_min_temp,...,min_min_temp,mean_sum_rain,max_sum_rain,min_sum_rain,mean_sun_time,max_sun_time,min_sun_time,mean_mean_humid,max_mean_humid,min_mean_humid
0,仙台,2004,11,11.256000,16.4,4.4,15.196000,19.3,7.5,7.152000,...,1.1,1.340000,20.0,0.0,4.436000,9.4,0.0,68.320000,93.0,45.0
1,仙台,2004,12,5.422581,13.3,-1.2,9.212903,21.6,0.4,1.825806,...,-4.4,2.048387,42.0,0.0,4.441935,8.9,0.0,64.225806,86.0,46.0
2,仙台,2005,1,1.590323,5.4,-2.0,5.077419,11.1,0.6,-1.809677,...,-5.6,2.000000,44.5,0.0,3.938710,9.4,0.0,69.709677,92.0,51.0
3,仙台,2005,2,0.828571,3.0,-1.3,4.335714,8.6,0.3,-2.221429,...,-4.7,2.000000,25.0,0.0,4.392857,9.0,0.0,68.750000,89.0,48.0
4,仙台,2005,3,4.061290,8.2,-1.2,8.332258,13.5,0.1,0.225806,...,-4.6,1.903226,28.0,0.0,5.522581,10.8,0.0,63.000000,89.0,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6907,鹿児島,2022,6,24.576667,29.3,20.7,28.130000,33.9,22.3,21.756667,...,16.6,16.216667,92.0,0.0,4.746667,13.0,0.0,81.633333,97.0,70.0
6908,鹿児島,2022,7,28.825000,30.5,26.4,32.725000,35.6,28.5,25.846875,...,23.5,12.953125,68.5,0.0,6.028125,13.5,0.0,78.562500,91.0,70.0
6909,鹿児島,2022,8,29.775000,30.7,28.3,34.240625,36.3,32.5,26.871875,...,24.3,6.187500,68.0,0.0,7.271875,12.7,1.8,76.937500,86.0,65.0
6910,鹿児島,2022,9,27.458065,30.6,23.7,31.270968,34.3,26.9,24.396774,...,19.4,9.225806,171.5,0.0,6.119355,11.0,0.0,75.322581,89.0,65.0


train.csv, test.csvには「各地」というエリアがあります。天候データ側ではこれを「全国」と読み替えることにして、全国平均を算出しておきます

In [11]:
agg_cols = [i for i in gb_df.columns if i not in ['year', 'month', 'area']]
tmp_df = gb_df.groupby(['year', 'month'])[agg_cols].agg(['mean']).reset_index()

new_cols = []
for col1, col2 in tmp_df.columns:
    new_cols.append(col1)

tmp_df.columns = new_cols
tmp_df['area'] = '全国'
tmp_df = tmp_df[gb_df.columns]
tmp_df

,area,year,month,mean_mean_temp,max_mean_temp,min_mean_temp,mean_max_temp,max_max_temp,min_max_temp,mean_min_temp,...,min_min_temp,mean_sum_rain,max_sum_rain,min_sum_rain,mean_sun_time,max_sun_time,min_sun_time,mean_mean_humid,max_mean_humid,min_mean_humid
0,全国,2004,11,13.046750,18.481250,8.406250,17.580375,22.950000,11.609375,9.046250,...,4.287500,3.456875,42.578125,0.0,5.446625,9.478125,0.00000,67.877500,87.96875,49.59375
1,全国,2004,12,8.011492,14.731250,1.662500,12.469456,21.259375,4.171875,4.086089,...,-1.271875,3.554940,54.796875,0.0,4.952319,9.059375,0.00000,65.214718,85.18750,48.43750
2,全国,2005,1,4.076210,7.975000,0.912500,8.123286,14.028125,3.909375,0.388609,...,-3.300000,2.198085,26.328125,0.0,4.467498,8.940625,0.00625,64.005040,84.96875,46.59375
3,全国,2005,2,4.199888,9.156250,0.143750,8.256920,14.600000,2.921875,0.549442,...,-3.390625,2.989955,25.125000,0.0,4.314951,9.321875,0.00000,63.796875,88.56250,46.25000
4,全国,2005,3,7.202218,12.068750,1.228125,11.891431,18.268750,4.409375,2.865927,...,-2.656250,3.122984,29.046875,0.0,5.416835,10.934375,0.00000,61.677419,87.09375,41.53125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,全国,2022,6,22.673542,29.337500,16.962500,27.130625,34.921875,18.940625,19.153542,...,14.009375,5.121875,48.750000,0.0,5.752917,13.109375,0.00000,76.471588,94.28125,57.37500
212,全国,2022,7,26.960254,30.100000,23.612500,31.261816,36.093750,25.478125,23.816699,...,21.446875,7.780924,71.359375,0.0,5.925195,12.975000,0.01875,78.466797,94.50000,63.37500
213,全国,2022,8,27.444727,30.506250,23.625000,31.780957,35.856250,26.640625,24.177148,...,19.412500,5.952889,55.296875,0.0,5.655469,12.203125,0.25000,77.232422,91.03125,61.62500
214,全国,2022,9,24.553528,28.603125,19.325000,28.913810,33.675000,23.078125,21.129536,...,15.471875,7.585181,80.500000,0.0,5.057359,11.390625,0.00000,77.218246,93.43750,61.00000


都道府県別と全国平均の値を結合します

In [15]:
wea_df = pd.concat([gb_df, tmp_df])

卸売データと天候データを結合するには、各々でエリアの名称が異なるため、エリアの対応をとる必要があります<br>
エリアの変換を行うmapを作成します

In [16]:
all_df = pd.concat([train_df, test_df])
all_df

,kind,date,amount,mode_price,area,year,weekno
0,だいこん,20051107,201445.0,735.0,千葉,2005,45
1,だいこん,20051108,189660.0,840.0,千葉_各地_青森,2005,45
2,だいこん,20051110,218166.0,735.0,千葉_各地_青森,2005,45
3,だいこん,20051111,182624.0,682.5,千葉_青森,2005,45
4,だいこん,20051112,220691.0,682.5,千葉_青森,2005,45
...,...,...,...,...,...,...,...
310,ミニトマト,20221124,NaN,NaN,各地_愛知,2022,47
311,ミニトマト,20221125,NaN,NaN,各地_愛知,2022,47
312,ミニトマト,20221126,NaN,NaN,各地_愛知,2022,47
313,ミニトマト,20221128,NaN,NaN,各地_愛知,2022,48


卸売データのエリアを列挙します

In [17]:
area_pairs = all_df['area'].unique()

yasai_areas = set()

for area_pair in area_pairs:
    areas = area_pair.split('_')
    yasai_areas |= set(areas)

yasai_areas

{'アメリカ',
 'カナダ',
 'トンガ',
 'ニュージーランド',
 'メキシコ',
 '中国',
 '佐賀',
 '兵庫',
 '北海道',
 '千葉',
 '各地',
 '和歌山',
 '埼玉',
 '宮城',
 '宮崎',
 '山形',
 '山梨',
 '岩手',
 '徳島',
 '愛媛',
 '愛知',
 '新潟',
 '栃木',
 '沖縄',
 '熊本',
 '神奈川',
 '福岡',
 '福島',
 '秋田',
 '群馬',
 '茨城',
 '長崎',
 '長野',
 '青森',
 '静岡',
 '香川',
 '高知',
 '鹿児島'}

天候データのエリアを列挙します

In [18]:
wea_areas = wea_df['area'].unique()
wea_areas

array(['仙台', '佐賀', '前橋', '千葉', '名古屋', '和歌山', '宇都宮', '宮崎', '山形', '帯広',
       '徳島', '新潟', '東京', '松山', '横浜', '水戸', '浜松', '熊本', '熊谷', '甲府', '盛岡',
       '神戸', '福岡', '福島', '秋田', '那覇', '長崎', '長野', '青森', '高松', '高知', '鹿児島',
       '全国'], dtype=object)

エリア変換のmapを作ります

In [19]:
area_map = {}

update_area_map = {
    '岩手':'盛岡','宮城':'仙台','静岡':'浜松','沖縄':'那覇','神奈川':'横浜','愛知':'名古屋','茨城':'水戸','北海道':'帯広','各地':'全国',
    '兵庫':'神戸','香川':'高松','埼玉':'熊谷','国内':'全国','山梨':'甲府','栃木':'宇都宮','群馬':'前橋','愛媛':'松山'
}

for yasai_area in yasai_areas:
    if yasai_area not in wea_areas and yasai_area not in update_area_map:
        area_map[yasai_area] = '全国' # 外国の天候は全国にしておく
    else:
        area_map[yasai_area] = yasai_area

area_map = {**area_map, **update_area_map}
area_map

{'アメリカ': '全国',
 '新潟': '新潟',
 '熊本': '熊本',
 '秋田': '秋田',
 '千葉': '千葉',
 '長崎': '長崎',
 '宮崎': '宮崎',
 '山形': '山形',
 '北海道': '帯広',
 '香川': '高松',
 '高知': '高知',
 'トンガ': '全国',
 '兵庫': '神戸',
 'ニュージーランド': '全国',
 '神奈川': '横浜',
 '茨城': '水戸',
 '鹿児島': '鹿児島',
 '栃木': '宇都宮',
 '宮城': '仙台',
 '青森': '青森',
 '和歌山': '和歌山',
 '長野': '長野',
 '静岡': '浜松',
 '埼玉': '熊谷',
 'カナダ': '全国',
 '各地': '全国',
 'メキシコ': '全国',
 '中国': '全国',
 '徳島': '徳島',
 '佐賀': '佐賀',
 '山梨': '甲府',
 '福島': '福島',
 '愛媛': '松山',
 '福岡': '福岡',
 '沖縄': '那覇',
 '愛知': '名古屋',
 '岩手': '盛岡',
 '群馬': '前橋',
 '国内': '全国'}

卸売データのエリアを天候データのエリアに変換します

In [20]:
all_df['area'] = all_df['area'].apply(lambda x: '_'.join([area_map[i] for i in x.split('_')]))
all_df

,kind,date,amount,mode_price,area,year,weekno
0,だいこん,20051107,201445.0,735.0,千葉,2005,45
1,だいこん,20051108,189660.0,840.0,千葉_全国_青森,2005,45
2,だいこん,20051110,218166.0,735.0,千葉_全国_青森,2005,45
3,だいこん,20051111,182624.0,682.5,千葉_青森,2005,45
4,だいこん,20051112,220691.0,682.5,千葉_青森,2005,45
...,...,...,...,...,...,...,...
310,ミニトマト,20221124,NaN,NaN,全国_名古屋,2022,47
311,ミニトマト,20221125,NaN,NaN,全国_名古屋,2022,47
312,ミニトマト,20221126,NaN,NaN,全国_名古屋,2022,47
313,ミニトマト,20221128,NaN,NaN,全国_名古屋,2022,48


In [21]:
test_df = all_df.iloc[train_df.shape[0]:]
train_df = all_df.iloc[:train_df.shape[0]]

ここまで加工したデータを出力します（出力せず、このまま学習・推論を行っても構いません）

In [22]:
test_df.to_csv('tutorial_preprocessed_test.csv', index=False)
train_df.to_csv('tutorial_preprocessed_train.csv', index=False)

一方で天候データについても、卸売データと単純にmergeするには加工が必要です<br>「千葉_全国_青森」のように複数エリアに跨っている場合の各統計量について、各エリアの平均を取ることにします

In [23]:
area_pairs = all_df['area'].unique()

In [24]:
target_cols = [i for i in wea_df.columns if i != 'area']

In [26]:
area_pair_dfs = []

for area_pair in area_pairs:
    areas = area_pair.split('_')
    if len(areas) > 0:
        area = areas[0]
        base_tmp_df = wea_df[wea_df['area'] == area]
        base_tmp_df = base_tmp_df[target_cols].reset_index(drop=True)
        for area in areas[1:]:
            tmp_df = wea_df[wea_df['area'] == area]
            tmp_df = tmp_df[target_cols].reset_index(drop=True)
            base_tmp_df = base_tmp_df.add(tmp_df)
        base_tmp_df /= len(areas)
        base_tmp_df['area'] = area_pair
        area_pair_dfs.append(base_tmp_df)

In [27]:
area_pair_df = pd.concat(area_pair_dfs)
print(area_pair_df.shape)
area_pair_df.head()

(46872, 21)


,year,month,mean_mean_temp,max_mean_temp,min_mean_temp,mean_max_temp,max_max_temp,min_max_temp,mean_min_temp,max_min_temp,...,mean_sum_rain,max_sum_rain,min_sum_rain,mean_sun_time,max_sun_time,min_sun_time,mean_mean_humid,max_mean_humid,min_mean_humid,area
0,2004.0,11.0,14.596000,19.3,9.6,18.340000,22.2,12.4,11.228000,17.4,...,5.840000,87.0,0.0,5.084000,9.5,0.0,71.760000,93.0,43.0,千葉
1,2004.0,12.0,9.641935,19.0,2.2,13.564516,24.3,3.8,6.138710,11.5,...,2.435484,26.0,0.0,5.509677,9.4,0.0,58.612903,86.0,37.0,千葉
2,2005.0,1.0,6.277419,13.1,3.1,10.461290,17.2,5.2,2.522581,6.4,...,3.338710,66.5,0.0,6.612903,9.8,0.0,54.838710,89.0,32.0,千葉
3,2005.0,2.0,5.960714,12.0,3.3,9.864286,15.4,5.1,2.060714,4.6,...,1.910714,11.5,0.0,5.225000,9.7,0.0,55.714286,93.0,31.0,千葉
4,2005.0,3.0,8.800000,13.7,1.6,13.122581,18.4,3.6,4.596774,10.0,...,2.612903,22.5,0.0,5.529032,11.3,0.0,59.129032,91.0,31.0,千葉


ここまで加工したデータを出力します（出力せず、このまま学習・推論を行っても構いません）

In [28]:
area_pair_df.to_csv('tutorial_preprocessed_weather.csv', index=False)

In [29]:
area_pair_df

,year,month,mean_mean_temp,max_mean_temp,min_mean_temp,mean_max_temp,max_max_temp,min_max_temp,mean_min_temp,max_min_temp,...,mean_sum_rain,max_sum_rain,min_sum_rain,mean_sun_time,max_sun_time,min_sun_time,mean_mean_humid,max_mean_humid,min_mean_humid,area
0,2004.0,11.0,14.596000,19.30,9.60,18.340000,22.20,12.40,11.228000,17.40,...,5.840000,87.00,0.0,5.084000,9.50,0.0,71.760000,93.0,43.0,千葉
1,2004.0,12.0,9.641935,19.00,2.20,13.564516,24.30,3.80,6.138710,11.50,...,2.435484,26.00,0.0,5.509677,9.40,0.0,58.612903,86.0,37.0,千葉
2,2005.0,1.0,6.277419,13.10,3.10,10.461290,17.20,5.20,2.522581,6.40,...,3.338710,66.50,0.0,6.612903,9.80,0.0,54.838710,89.0,32.0,千葉
3,2005.0,2.0,5.960714,12.00,3.30,9.864286,15.40,5.10,2.060714,4.60,...,1.910714,11.50,0.0,5.225000,9.70,0.0,55.714286,93.0,31.0,千葉
4,2005.0,3.0,8.800000,13.70,1.60,13.122581,18.40,3.60,4.596774,10.00,...,2.612903,22.50,0.0,5.529032,11.30,0.0,59.129032,91.0,31.0,千葉
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,2022.0,6.0,21.523333,28.70,16.05,26.713333,34.60,18.40,17.646667,24.50,...,4.183333,61.00,0.0,6.455000,13.35,0.0,73.783333,87.5,56.5,新潟_長野
212,2022.0,7.0,26.151563,29.10,23.70,31.015625,35.40,26.40,22.543750,24.60,...,5.726562,61.50,0.0,7.187500,13.75,0.0,77.640625,91.5,64.0,新潟_長野
213,2022.0,8.0,26.235937,29.55,21.10,30.957812,35.45,24.75,22.604688,25.45,...,6.609375,38.75,0.0,5.126563,11.90,0.0,78.031250,91.0,67.0,新潟_長野
214,2022.0,9.0,23.167742,29.40,16.90,27.770968,34.40,20.60,19.619355,25.70,...,6.427419,50.25,0.0,5.600000,11.90,0.0,76.951613,95.0,59.0,新潟_長野


In [30]:
test_df

,kind,date,amount,mode_price,area,year,weekno
0,だいこん,20221101,NaN,NaN,千葉_全国_横浜,2022,44
1,だいこん,20221102,NaN,NaN,千葉_全国_横浜,2022,44
2,だいこん,20221104,NaN,NaN,千葉_全国_横浜,2022,44
3,だいこん,20221105,NaN,NaN,千葉_全国_横浜,2022,44
4,だいこん,20221107,NaN,NaN,千葉_全国_横浜,2022,45
...,...,...,...,...,...,...,...
310,ミニトマト,20221124,NaN,NaN,全国_名古屋,2022,47
311,ミニトマト,20221125,NaN,NaN,全国_名古屋,2022,47
312,ミニトマト,20221126,NaN,NaN,全国_名古屋,2022,47
313,ミニトマト,20221128,NaN,NaN,全国_名古屋,2022,48


In [31]:
train_df

,kind,date,amount,mode_price,area,year,weekno
0,だいこん,20051107,201445.0,735.0,千葉,2005,45
1,だいこん,20051108,189660.0,840.0,千葉_全国_青森,2005,45
2,だいこん,20051110,218166.0,735.0,千葉_全国_青森,2005,45
3,だいこん,20051111,182624.0,682.5,千葉_青森,2005,45
4,だいこん,20051112,220691.0,682.5,千葉_青森,2005,45
...,...,...,...,...,...,...,...
77746,しめじ,20170526,17231.0,48.5,新潟_長野,2017,21
77747,しめじ,20170527,17105.0,48.5,新潟_長野,2017,21
77748,しめじ,20170529,12406.0,48.5,新潟_長野,2017,22
77749,しめじ,20170530,12320.0,48.5,新潟_長野,2017,22
